# Robo movement
Creates a robot movement to generate a dataset from the lidars 

In [1]:
import time
import numpy as np
import py_at_broker as pab
from IPython.display import clear_output
import math as math

import pandas as pd
import pylab as pl
from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline
b = pab.broker()


%load_ext autoreload
%autoreload 2
from toolbox import kinematic_mapping as km
from toolbox import data_processing as dp
from toolbox import movementtoolbox as mt


In [2]:
b.register_signal('franka_target_pos', pab.MsgType.target_pos)

True

In [3]:
b.request_signal("franka_state", pab.MsgType.franka_state, True)


True

In [11]:
fn = 'real_robot_spiral.p'

In [15]:
def generate_random(): 
    infeasible = True
    border_x = .4
    border_y = .3
    border_z = .8
    i = 0
    while infeasible:
        mu_x = 0.2
        mu_y = 0
        mu_z = 0.4

        sig_x = 0.15
        sig_y = 0.3
        sig_z = 0.3

        x, y, z = np.random.multivariate_normal([mu_x, mu_y, mu_z], np.diag([sig_x, sig_y, sig_z]))
        i += 1 
        if (x> 0) and (x < border_x) and (abs(y) < border_y) and (z> 0.05) and (z < border_z) : 
            infeasible = False
    return x, y, z
    #print(x, y, z)
    #print('iterations: {}'.format(i))

In [4]:
sleep_time = 0.5
go_time = 0.15

time.sleep(sleep_time)
#x=0.5

target_msg, current_counter= mt.create_message()

b.send_msg('franka_target_pos', target_msg)
time.sleep(sleep_time)


msg = b.recv_msg("franka_state", 0)
print(msg.get_fnumber())


inside_range = True
i = 10
polarity = 1
scaling=1.2


while inside_range : 
    timeth=i/scaling


    #y, z = mt.create_spiral(timeth, 0.005)
    x, y, z, = generate_random()
    pos = np.array([x, y, z+0.7])
    frame=current_counter+1
    target_msg, current_counter = mt.create_message(counter=frame, pos = pos, go_time=go_time*0.3*i)

   
    
     
    clear_output(wait = True)
    print(pos)
    print(polarity)
    b.send_msg('franka_target_pos', target_msg)
    time.sleep(sleep_time)
    i += polarity
    
    if z > 0.2: 
        polarity = -1
    elif i == 10: 
        polarity = 1



[ 0.5        -0.09656212  0.62718894]
-1


KeyboardInterrupt: 

0.5013503286834864 -0.042021417829258965 0.45153003794385066
iterations: 27
